In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, ShuffleSplit, GridSearchCV
from sklearn.metrics import roc_auc_score, roc_curve, auc
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier

import os
import time
import sys

1 -- Spruce/Fir
2 -- Lodgepole Pine
3 -- Ponderosa Pine
4 -- Cottonwood/Willow
5 -- Aspen
6 -- Douglas-fir
7 -- Krummholz

In [2]:
soilsCovDF = pd.read_csv("Data/soilsCovtype.csv")

/Users/Megan/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
soilsCovDF.head()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Cover_Type,SoilType,Soil_Type,code,family,sub-family,description,climatic zone,geologic zone,GeoSoils
0,2596,51,3,258,0,510,221,232,148,6279,...,5,29,29,7745,Como,Legault,extremely stony,subalpine,igneous and metamorphic,1
1,2590,56,2,212,-6,390,220,235,151,6225,...,5,29,29,7745,Como,Legault,extremely stony,subalpine,igneous and metamorphic,1
2,2595,45,2,153,-1,391,220,234,150,6172,...,5,29,29,7745,Como,Legault,extremely stony,subalpine,igneous and metamorphic,1
3,2579,132,6,300,-15,67,230,237,140,6031,...,2,29,29,7745,Como,Legault,extremely stony,subalpine,igneous and metamorphic,1
4,2606,45,7,270,5,633,222,225,138,6256,...,5,29,29,7745,Como,Legault,extremely stony,subalpine,igneous and metamorphic,1


Vast majority of trees are either Spruce/Fir(1) or Lodgepole Pine(2)

In [38]:
soilsCovDF.columns

Index(['Elevation', 'Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology',
       'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways',
       'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm',
       'Horizontal_Distance_To_Fire_Points', 'Wilderness_Area1',
       'Wilderness_Area2', 'Wilderness_Area3', 'Wilderness_Area4',
       'Cover_Type', 'SoilType', 'Soil_Type', 'code', 'family', 'sub-family',
       'description', 'climatic zone', 'geologic zone', 'GeoSoils'],
      dtype='object')

In [78]:
features_to_use = [
#     "Cover_Type",
    "GeoSoils",
    "Elevation",  
    "Slope", 
    "Hillshade_9am", 
    "Hillshade_3pm", 
    "Horizontal_Distance_To_Hydrology", 
    "Horizontal_Distance_To_Fire_Points"
]

In [41]:
CoverTypeDF = soilsCovDF.drop(columns=[
    "Aspect",
    "Vertical_Distance_To_Hydrology",
    "Horizontal_Distance_To_Roadways",
    "Hillshade_Noon",
    "Wilderness_Area1",
    "Wilderness_Area2",
    "Wilderness_Area3",
    "Wilderness_Area4",
    "SoilType",
    "Soil_Type",
    "code",
    "family",
    "sub-family",
    "description",
    "climatic zone",
    "geologic zone"
    ])

In [79]:
CoverTypeDF.head()

,Elevation,Slope,Horizontal_Distance_To_Hydrology,Hillshade_9am,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Cover_Type,GeoSoils,Y1,Y2,Y3,Y4,Y5,Y6,Y7
0,2596,3,258,221,148,6279,5,1,0,0,0,0,1,0,0
1,2590,2,212,220,151,6225,5,1,0,0,0,0,1,0,0
2,2595,2,153,220,150,6172,5,1,0,0,0,0,1,0,0
3,2579,6,300,230,140,6031,2,1,0,1,0,0,0,0,0
4,2606,7,270,222,138,6256,5,1,0,0,0,0,1,0,0


In [43]:
CoverTypeDF.dtypes

Elevation                             int64
Slope                                 int64
Horizontal_Distance_To_Hydrology      int64
Hillshade_9am                         int64
Hillshade_3pm                         int64
Horizontal_Distance_To_Fire_Points    int64
Cover_Type                            int64
GeoSoils                              int64
dtype: object

In [44]:
CoverTypeDF['Y1'] = CoverTypeDF.Cover_Type.apply(lambda x: 1 if x == 1 else 0)
CoverTypeDF['Y2'] = CoverTypeDF.Cover_Type.apply(lambda x: 1 if x == 2 else 0)
CoverTypeDF['Y3'] = CoverTypeDF.Cover_Type.apply(lambda x: 1 if x == 3 else 0)
CoverTypeDF['Y4'] = CoverTypeDF.Cover_Type.apply(lambda x: 1 if x == 4 else 0)
CoverTypeDF['Y5'] = CoverTypeDF.Cover_Type.apply(lambda x: 1 if x == 5 else 0)
CoverTypeDF['Y6'] = CoverTypeDF.Cover_Type.apply(lambda x: 1 if x == 6 else 0)
CoverTypeDF['Y7'] = CoverTypeDF.Cover_Type.apply(lambda x: 1 if x == 7 else 0)

In [234]:
CoverTypeDF.head()

CoverTypeCSV = CoverTypeDF.to_csv("Data/CovTypeDF.csv")

In [48]:
X = CoverTypeDF[features_to_use]

In [86]:
X.head()

,GeoSoils,Elevation,Slope,Hillshade_9am,Hillshade_3pm,Horizontal_Distance_To_Hydrology,Horizontal_Distance_To_Fire_Points
0,1,2596,3,221,148,258,6279
1,1,2590,2,220,151,212,6225
2,1,2595,2,220,150,153,6172
3,1,2579,6,230,140,300,6031
4,1,2606,7,222,138,270,6256


In [87]:
X_encoded = pd.get_dummies(X)

In [88]:
y = CoverTypeDF[["Cover_Type"]]

In [143]:
X_train, X_test, y_train, y_test = train_test_split(X_encoded[features_to_use], y)

In [145]:
# normalize
sds = StandardScaler()
sds.fit(X_train)
X_train = sds.transform(X_train)
X_test = sds.transform(X_test)

In [146]:
for i in range(5):
    
    t0 = time.time()
    clf = DecisionTreeClassifier()
    
    clf.fit(X_train, y_train)
    t1 = time.time()
    
    ypred = clf.predict(X_test)
    scores = roc_auc_score(y_test, ypred)
    
    print(f"Decision tree {i+1} took {t1-t0} seconds to train and got a score of {scores}")
    print()
    print("And here are the top features: ")
    c = X_encoded.columns 
    for cfi in zip(c, clf.feature_importances_):
        print(cfi)
    print()

ValueError: multiclass format is not supported

#### After trying to run the Decision Tree Classifier on main Dataframe - the results come back with "ValueError: multiclass format is not supported". Now is the time to separate Cover_Type out into separate columns with values of '1' or '0' in order to allow for binary values for each covertype and re-run for each.

In [91]:
features_to_use = [
#     "Cover_Type",
    "GeoSoils",
    "Elevation",  
    "Slope", 
    "Hillshade_9am", 
    "Hillshade_3pm", 
    "Horizontal_Distance_To_Hydrology", 
    "Horizontal_Distance_To_Fire_Points",
]

In [92]:
X = CoverTypeDF[features_to_use]

In [93]:
y1 = CoverTypeDF[["Y1"]]

In [94]:
X.head()

,GeoSoils,Elevation,Slope,Hillshade_9am,Hillshade_3pm,Horizontal_Distance_To_Hydrology,Horizontal_Distance_To_Fire_Points
0,1,2596,3,221,148,258,6279
1,1,2590,2,220,151,212,6225
2,1,2595,2,220,150,153,6172
3,1,2579,6,230,140,300,6031
4,1,2606,7,222,138,270,6256


In [95]:
y1.head()

,Y1
0,0
1,0
2,0
3,0
4,0


In [147]:
X_encoded = pd.get_dummies(X)

In [148]:
X_train1, X_test1, y_train1, y_test1 = train_test_split(X_encoded, y1)

In [149]:
X_train1.shape, X_test1.shape

((435759, 7), (145253, 7))

In [150]:
X_train1.head()

,GeoSoils,Elevation,Slope,Hillshade_9am,Hillshade_3pm,Horizontal_Distance_To_Hydrology,Horizontal_Distance_To_Fire_Points
177269,1,3136,1,220,154,95,2431
417564,1,2973,6,202,175,942,2147
300492,0,3191,12,237,130,216,2538
133357,1,2875,11,239,119,560,2896
467667,1,3261,16,246,103,240,3520


In [151]:
pd.concat([X_train1, y_train1], axis=1).head()

,GeoSoils,Elevation,Slope,Hillshade_9am,Hillshade_3pm,Horizontal_Distance_To_Hydrology,Horizontal_Distance_To_Fire_Points,Y1
177269,1,3136,1,220,154,95,2431,1
417564,1,2973,6,202,175,942,2147,0
300492,0,3191,12,237,130,216,2538,1
133357,1,2875,11,239,119,560,2896,0
467667,1,3261,16,246,103,240,3520,0


In [152]:
sds1 = StandardScaler()
sds1.fit(X_train1)
X_train1 = sds1.transform(X_train1)
X_test1 = sds1.transform(X_test1)

## Decision Tree Classifier

#### For Type 1 Cover-types
1 -- Spruce/Fir
2 -- Lodgepole Pine
3 -- Ponderosa Pine
4 -- Cottonwood/Willow
5 -- Aspen
6 -- Douglas-fir
7 -- Krummholz

In [153]:
for i in range(5):
    
    t0 = time.time()
    clf = DecisionTreeClassifier()
    
    clf.fit(X_train1, y_train1)
    t1 = time.time()
    
    ypred1 = clf.predict(X_test1)
    scores1 = roc_auc_score(y_test1, ypred1)
    
    print(f"Decision tree {i+1} took {t1-t0} seconds to train and got a score of {scores1}")
    print()
    print("And here are the top features: ")
    c = X_encoded.columns 
    for cfi in zip(c, clf.feature_importances_):
        print(cfi)
    print()

Decision tree 1 took 2.7865147590637207 seconds to train and got a score of 0.8740634402684034

And here are the top features: 
('GeoSoils', 0.03101693052360258)
('Elevation', 0.38717314971514966)
('Slope', 0.06383199953894321)
('Hillshade_9am', 0.08458628527373345)
('Hillshade_3pm', 0.09216686856236657)
('Horizontal_Distance_To_Hydrology', 0.11885553248176757)
('Horizontal_Distance_To_Fire_Points', 0.22236923390443705)

Decision tree 2 took 2.847913980484009 seconds to train and got a score of 0.8736972762470716

And here are the top features: 
('GeoSoils', 0.031042389207160414)
('Elevation', 0.3868333530330626)
('Slope', 0.06400798104907697)
('Hillshade_9am', 0.08405311237498957)
('Hillshade_3pm', 0.09234285577930298)
('Horizontal_Distance_To_Hydrology', 0.11857087156984947)
('Horizontal_Distance_To_Fire_Points', 0.22314943698655795)

Decision tree 3 took 3.5517187118530273 seconds to train and got a score of 0.8735739261012566

And here are the top features: 
('GeoSoils', 0.03102829

#### For Type 2 Covertypes

In [154]:
y2 = CoverTypeDF[["Y2"]]
X_train2, X_test2, y_train2, y_test2 = train_test_split(X_encoded, y2)
pd.concat([X_train2, y_train2], axis=1)

# normalizer
sds2 = StandardScaler()
sds2.fit(X_train2)
X_train2 = sds2.transform(X_train2)
X_test2 = sds2.transform(X_test2)

for i in range(5):
    
    t0 = time.time()
    clf = DecisionTreeClassifier()
    
    clf.fit(X_train2, y_train2)
    t1 = time.time()
    
    ypred2 = clf.predict(X_test2)
    scores2 = roc_auc_score(y_test2, ypred2)
    
    print(f"Decision tree {i+1} took {t1-t0} seconds to train and got a score of {scores2}")
    print()
    print("And here are the top features: ")
    c = X_encoded.columns 
    for cfi in zip(c, clf.feature_importances_):
        print(cfi)
    print()

Decision tree 1 took 3.7406880855560303 seconds to train and got a score of 0.8630779100011052

And here are the top features: 
('GeoSoils', 0.026637025833296663)
('Elevation', 0.39285896156886146)
('Slope', 0.06438275513439441)
('Hillshade_9am', 0.08780546818298143)
('Hillshade_3pm', 0.0925730050733032)
('Horizontal_Distance_To_Hydrology', 0.1197585385918468)
('Horizontal_Distance_To_Fire_Points', 0.215984245615316)

Decision tree 2 took 3.4783151149749756 seconds to train and got a score of 0.8631008018236311

And here are the top features: 
('GeoSoils', 0.026558079801189845)
('Elevation', 0.39261783373596526)
('Slope', 0.0642397865771332)
('Hillshade_9am', 0.08832777889726404)
('Hillshade_3pm', 0.09206278076028586)
('Horizontal_Distance_To_Hydrology', 0.11988606418102396)
('Horizontal_Distance_To_Fire_Points', 0.21630767604713777)

Decision tree 3 took 3.385528802871704 seconds to train and got a score of 0.8622819165389309

And here are the top features: 
('GeoSoils', 0.02663622951

In [156]:
y3 = CoverTypeDF[["Y3"]]
X_train3, X_test3, y_train3, y_test3 = train_test_split(X_encoded, y3)
pd.concat([X_train3, y_train3], axis=1)

# normalizer
sds3 = StandardScaler()
sds3.fit(X_train3)
X_train3 = sds3.transform(X_train3)
X_test3 = sds3.transform(X_test3)

for i in range(5):
    
    t0 = time.time()
    clf = DecisionTreeClassifier()
    
    clf.fit(X_train3, y_train3)
    t1 = time.time()
    
    ypred3 = clf.predict(X_test3)
    scores3 = roc_auc_score(y_test3, ypred3)
    
    print(f"Decision tree {i+1} took {t1-t0} seconds to train and got a score of {scores3}")
    print()
    print("And here are the top features: ")
    c = X_encoded.columns 
    for cfi in zip(c, clf.feature_importances_):
        print(cfi)
    print()

Decision tree 1 took 1.5345330238342285 seconds to train and got a score of 0.8962780250075763

And here are the top features: 
('GeoSoils', 0.01080194015035082)
('Elevation', 0.5230611165247894)
('Slope', 0.06120214853610019)
('Hillshade_9am', 0.0843645787447023)
('Hillshade_3pm', 0.07229082851474848)
('Horizontal_Distance_To_Hydrology', 0.09809239527263823)
('Horizontal_Distance_To_Fire_Points', 0.15018699225667065)

Decision tree 2 took 1.8515489101409912 seconds to train and got a score of 0.8978997488241305

And here are the top features: 
('GeoSoils', 0.010722735091608756)
('Elevation', 0.5230922074433536)
('Slope', 0.06170549964309358)
('Hillshade_9am', 0.08499825282772859)
('Hillshade_3pm', 0.0727213751277202)
('Horizontal_Distance_To_Hydrology', 0.09716870958082392)
('Horizontal_Distance_To_Fire_Points', 0.1495912202856713)

Decision tree 3 took 1.5599498748779297 seconds to train and got a score of 0.8976564105720007

And here are the top features: 
('GeoSoils', 0.01078981484

In [157]:
y4 = CoverTypeDF[["Y4"]]
X_train4, X_test4, y_train4, y_test4 = train_test_split(X_encoded, y4)
pd.concat([X_train4, y_train4], axis=1)

# normalizer
sds4 = StandardScaler()
sds4.fit(X_train4)
X_train4 = sds4.transform(X_train4)
X_test4 = sds4.transform(X_test4)

for i in range(5):
    
    t0 = time.time()
    clf = DecisionTreeClassifier()
    
    clf.fit(X_train4, y_train4)
    t1 = time.time()
    
    ypred4 = clf.predict(X_test4)
    scores4 = roc_auc_score(y_test4, ypred4)
    
    print(f"Decision tree {i+1} took {t1-t0} seconds to train and got a score of {scores4}")
    print()
    print("And here are the top features: ")
    c = X_encoded.columns 
    for cfi in zip(c, clf.feature_importances_):
        print(cfi)
    print()

Decision tree 1 took 0.88201904296875 seconds to train and got a score of 0.8723785880250433

And here are the top features: 
('GeoSoils', 0.01984288321041823)
('Elevation', 0.25517658500863316)
('Slope', 0.04979410447008007)
('Hillshade_9am', 0.14334063642691464)
('Hillshade_3pm', 0.0977250680848111)
('Horizontal_Distance_To_Hydrology', 0.24221501303042364)
('Horizontal_Distance_To_Fire_Points', 0.19190570976871915)

Decision tree 2 took 1.0018119812011719 seconds to train and got a score of 0.8645806286439097

And here are the top features: 
('GeoSoils', 0.02046373359782215)
('Elevation', 0.25322250279941944)
('Slope', 0.05052242427691673)
('Hillshade_9am', 0.147054771258858)
('Hillshade_3pm', 0.09083338385292836)
('Horizontal_Distance_To_Hydrology', 0.2428644017402608)
('Horizontal_Distance_To_Fire_Points', 0.19503878247379453)

Decision tree 3 took 0.885662317276001 seconds to train and got a score of 0.8680852192316983

And here are the top features: 
('GeoSoils', 0.02142508227469

In [158]:
y5 = CoverTypeDF[["Y5"]]
X_train5, X_test5, y_train5, y_test5 = train_test_split(X_encoded, y5)
pd.concat([X_train5, y_train5], axis=1)

# normalizer
sds5 = StandardScaler()
sds5.fit(X_train5)
X_train5 = sds5.transform(X_train5)
X_test5 = sds5.transform(X_test5)

for i in range(5):
    
    t0 = time.time()
    clf = DecisionTreeClassifier()
    
    clf.fit(X_train5, y_train5)
    t1 = time.time()
    
    ypred5 = clf.predict(X_test5)
    scores5 = roc_auc_score(y_test5, ypred5)
    
    print(f"Decision tree {i+1} took {t1-t0} seconds to train and got a score of {scores5}")
    print()
    print("And here are the top features: ")
    c = X_encoded.columns 
    for cfi in zip(c, clf.feature_importances_):
        print(cfi)
    print()

Decision tree 1 took 1.5002822875976562 seconds to train and got a score of 0.7992781484410958

And here are the top features: 
('GeoSoils', 0.013724169074764904)
('Elevation', 0.23394100958833672)
('Slope', 0.07070461653654218)
('Hillshade_9am', 0.1074835604725528)
('Hillshade_3pm', 0.14824911828700835)
('Horizontal_Distance_To_Hydrology', 0.16995364544404049)
('Horizontal_Distance_To_Fire_Points', 0.25594388059675455)

Decision tree 2 took 1.4597291946411133 seconds to train and got a score of 0.8013842241685428

And here are the top features: 
('GeoSoils', 0.014013902524219794)
('Elevation', 0.23657188726857584)
('Slope', 0.07015938960983353)
('Hillshade_9am', 0.10262774258912132)
('Hillshade_3pm', 0.14930872056706282)
('Horizontal_Distance_To_Hydrology', 0.1734572314318478)
('Horizontal_Distance_To_Fire_Points', 0.253861126009339)

Decision tree 3 took 1.641028881072998 seconds to train and got a score of 0.8001177790036482

And here are the top features: 
('GeoSoils', 0.0134772451

In [159]:
y6 = CoverTypeDF[["Y6"]]
X_train6, X_test6, y_train6, y_test6 = train_test_split(X_encoded, y6)
pd.concat([X_train6, y_train6], axis=1)

# normalizer
sds6 = StandardScaler()
sds6.fit(X_train6)
X_train6 = sds6.transform(X_train6)
X_test6 = sds6.transform(X_test6)

for i in range(5):
    
    t0 = time.time()
    clf = DecisionTreeClassifier()
    
    clf.fit(X_train6, y_train6)
    t1 = time.time()
    
    ypred6 = clf.predict(X_test6)
    scores6 = roc_auc_score(y_test6, ypred6)
    
    print(f"Decision tree {i+1} took {t1-t0} seconds to train and got a score of {scores6}")
    print()
    print("And here are the top features: ")
    c = X_encoded.columns 
    for cfi in zip(c, clf.feature_importances_):
        print(cfi)
    print()

Decision tree 1 took 2.178934097290039 seconds to train and got a score of 0.8361524742310668

And here are the top features: 
('GeoSoils', 0.01325293196524422)
('Elevation', 0.3658225354616766)
('Slope', 0.08039923897062748)
('Hillshade_9am', 0.11090541404328044)
('Hillshade_3pm', 0.10333012793835496)
('Horizontal_Distance_To_Hydrology', 0.13186365063484376)
('Horizontal_Distance_To_Fire_Points', 0.19442610098597268)

Decision tree 2 took 2.1755990982055664 seconds to train and got a score of 0.8354797988688833

And here are the top features: 
('GeoSoils', 0.013493465590915093)
('Elevation', 0.36533189044929526)
('Slope', 0.08005138510194366)
('Hillshade_9am', 0.11175497379244324)
('Hillshade_3pm', 0.10315968049797587)
('Horizontal_Distance_To_Hydrology', 0.13154333530167325)
('Horizontal_Distance_To_Fire_Points', 0.19466526926575367)

Decision tree 3 took 2.224271059036255 seconds to train and got a score of 0.836307161119111

And here are the top features: 
('GeoSoils', 0.0137114182

In [160]:
y7 = CoverTypeDF[["Y7"]]
X_train7, X_test7, y_train7, y_test7 = train_test_split(X_encoded, y7)
pd.concat([X_train7, y_train7], axis=1)

# normalizer
sds7 = StandardScaler()
sds7.fit(X_train7)
X_train7 = sds7.transform(X_train7)
X_test7 = sds7.transform(X_test7)

for i in range(5):
    
    t0 = time.time()
    clf = DecisionTreeClassifier()
    
    clf.fit(X_train7, y_train7)
    t1 = time.time()
    
    ypred7 = clf.predict(X_test7)
    scores7 = roc_auc_score(y_test7, ypred7)
    
    print(f"Decision tree {i+1} took {t1-t0} seconds to train and got a score of {scores7}")
    print()
    print("And here are the top features: ")
    c = X_encoded.columns 
    for cfi in zip(c, clf.feature_importances_):
        print(cfi)
    print()

Decision tree 1 took 2.1903350353240967 seconds to train and got a score of 0.9169524910935791

And here are the top features: 
('GeoSoils', 0.01058868323197978)
('Elevation', 0.38819911808902835)
('Slope', 0.07039893847886328)
('Hillshade_9am', 0.08289096722620014)
('Hillshade_3pm', 0.09191642807746743)
('Horizontal_Distance_To_Hydrology', 0.11837097545860513)
('Horizontal_Distance_To_Fire_Points', 0.23763488943785588)

Decision tree 2 took 2.432262897491455 seconds to train and got a score of 0.9157488853550937

And here are the top features: 
('GeoSoils', 0.010438253799851088)
('Elevation', 0.3878306074067173)
('Slope', 0.070954258740355)
('Hillshade_9am', 0.08351559148856595)
('Hillshade_3pm', 0.0931393304751324)
('Horizontal_Distance_To_Hydrology', 0.11853639819453055)
('Horizontal_Distance_To_Fire_Points', 0.23558555989484764)

Decision tree 3 took 2.1804537773132324 seconds to train and got a score of 0.9145988311546084

And here are the top features: 
('GeoSoils', 0.01085542549

#### For both all Covertypes, the lowest CLF score for features is consistently GeoSoils. Now it is time to take it out of the 'features_to_use' set to see if this helps to improve the model.

In [177]:
features_to_use = [
#     "Cover_Type",
#     "GeoSoils",
    "Elevation",  
    "Slope", 
    "Hillshade_9am", 
    "Hillshade_3pm", 
    "Horizontal_Distance_To_Hydrology", 
    "Horizontal_Distance_To_Fire_Points"
]

In [178]:
features_to_use

['Elevation',
 'Slope',
 'Hillshade_9am',
 'Hillshade_3pm',
 'Horizontal_Distance_To_Hydrology',
 'Horizontal_Distance_To_Fire_Points']

#### Re-run Y1 through Y7 without GeoSoils

In [179]:
X = CoverTypeDF[features_to_use]
X_encoded = pd.get_dummies(X)

In [180]:
X.head(2)

,Elevation,Slope,Hillshade_9am,Hillshade_3pm,Horizontal_Distance_To_Hydrology,Horizontal_Distance_To_Fire_Points
0,2596,3,221,148,258,6279
1,2590,2,220,151,212,6225


In [181]:
y1 = CoverTypeDF[["Y1"]]
X_train1, X_test1, y_train1, y_test1 = train_test_split(X_encoded, y1)
pd.concat([X_train1, y_train1], axis=1)

# normalizer
sds1 = StandardScaler()
sds1.fit(X_train1)
X_train1 = sds1.transform(X_train1)
X_test1 = sds1.transform(X_test1)

for i in range(5):
    
    t0 = time.time()
    clf = DecisionTreeClassifier()
    
    clf.fit(X_train1, y_train1)
    t1 = time.time()
    
    ypred1 = clf.predict(X_test1)
    scores1 = roc_auc_score(y_test1, ypred1)
    
    print(f"Decision tree {i+1} took {t1-t0} seconds to train and got a score of {scores1}")
    print()
    print("And here are the top features: ")
    c = X_encoded.columns 
    for cfi in zip(c, clf.feature_importances_):
        print(cfi)
    print()

Decision tree 1 took 3.127523183822632 seconds to train and got a score of 0.8591156521773544

And here are the top features: 
('Elevation', 0.39500321247033704)
('Slope', 0.07086760749171857)
('Hillshade_9am', 0.09376944860341774)
('Hillshade_3pm', 0.09719249591738567)
('Horizontal_Distance_To_Hydrology', 0.11506848240904193)
('Horizontal_Distance_To_Fire_Points', 0.228098753108099)

Decision tree 2 took 2.8862929344177246 seconds to train and got a score of 0.8590093320590863

And here are the top features: 
('Elevation', 0.3946620029047584)
('Slope', 0.07056534335059951)
('Hillshade_9am', 0.09394708154282107)
('Hillshade_3pm', 0.0971732486134982)
('Horizontal_Distance_To_Hydrology', 0.11595217281070778)
('Horizontal_Distance_To_Fire_Points', 0.22770015077761513)

Decision tree 3 took 2.911633253097534 seconds to train and got a score of 0.8588732896910146

And here are the top features: 
('Elevation', 0.3943942349747434)
('Slope', 0.0710276379624719)
('Hillshade_9am', 0.093709284211

#### Taking GeoSoils out did not have much of an impact to the overall model, but did decrease the score slightly. So - taking this variable out did not improve the model. It appears that adding variables may help to improve the model.

In [ ]:
y7 = CoverTypeDF[["Y7"]]
X_train7, X_test7, y_train7, y_test7 = train_test_split(X_encoded, y7, random_state=35)

for i in range(5):
    
    t0 = time.time()
    clf = DecisionTreeClassifier()
    
    clf.fit(X_train7, y_train7)
    t1 = time.time()
    
    ypred7 = clf.predict(X_test7)
    scores7 = roc_auc_score(y_test7, ypred7)
    
    print(f"Decision tree {i+1} took {t1-t0} seconds to train and got a score of {scores7}")
    print()
    print("And here are the top features: ")
    c = X_encoded.columns 
    for cfi in zip(c, clf.feature_importances_):
        print(cfi)
    print()

## Model Bench and Model Selection

In [182]:
X_encoded[features_to_use].shape

(581012, 6)

In [223]:
features_to_use = [
#     "Cover_Type",
    "GeoSoils",
    "Elevation",  
    "Slope", 
    "Hillshade_9am", 
    "Hillshade_3pm", 
    "Horizontal_Distance_To_Hydrology", 
    "Horizontal_Distance_To_Fire_Points"
]

In [227]:
X = CoverTypeDF[features_to_use]
X_encoded = pd.get_dummies(X)
# X_encoded[features_to_use]

In [231]:
# CoverTypeDF.head()

CoverTypeDF_NP = CoverTypeDF.values

In [233]:
X_train, X_test, y_train, y_test = CoverTypeDF(X_encoded[features_to_use], y, random_state=35)

TypeError: 'DataFrame' object is not callable

In [ ]:
X_train.head()

In [214]:
def modelbench(classifier, X_train, Y_train):
    
    rs = ShuffleSplit(n_splits=3, test_size=.33)
    
    rs.get_n_splits(X_train)
    
    Y_train = Y_train.values
    Y_train = Y_train.ravel()

    for train_idx, test_idx in rs.split(X_train):
        ssX_train, ssX_test = X_train[train_idx], X_train[test_idx]
        ssy_train, ssy_test = Y_train[train_idx], Y_train[test_idx]

        t0 = time.time()
        
        clf = classifier
        clf.fit(X=ssX_train, y=ssy_train)

        ypred1 = clf.predict(ssX_test)

        scores1 = roc_auc_score(ssy_test, ypred)
    
        t1 = time.time()
    
        print(f"{classifier} \n took {t1-t0} seconds to run and got a score of {scores}")
        print()

In [202]:
X_encoded.head()

,Elevation,Slope,Hillshade_9am,Hillshade_3pm,Horizontal_Distance_To_Hydrology,Horizontal_Distance_To_Fire_Points
0,2596,3,221,148,258,6279
1,2590,2,220,151,212,6225
2,2595,2,220,150,153,6172
3,2579,6,230,140,300,6031
4,2606,7,222,138,270,6256


In [203]:
X_train1.head()

,Elevation,Slope,Hillshade_9am,Hillshade_3pm,Horizontal_Distance_To_Hydrology,Horizontal_Distance_To_Fire_Points
218641,2860,2,214,160,0,2317
188273,2955,9,207,156,0,904
45378,3185,5,213,155,484,4074
519086,2846,23,180,203,67,1739
429950,2861,17,207,127,190,2736


In [205]:
X_train.shape

(435759, 6)

In [207]:
y_train1.shape

(435759, 1)

In [208]:
y_train1.head()

,Y1
218641,1
188273,1
45378,1
519086,0
429950,1


In [206]:
y_test1.shape

(145253, 1)

In [216]:
CoverTypeDF.head()
CoverTypeDF.to_csv(index=False)

,Elevation,Slope,Horizontal_Distance_To_Hydrology,Hillshade_9am,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Cover_Type,GeoSoils,Y1,Y2,Y3,Y4,Y5,Y6,Y7
0,2596,3,258,221,148,6279,5,1,0,0,0,0,1,0,0
1,2590,2,212,220,151,6225,5,1,0,0,0,0,1,0,0
2,2595,2,153,220,150,6172,5,1,0,0,0,0,1,0,0
3,2579,6,300,230,140,6031,2,1,0,1,0,0,0,0,0
4,2606,7,270,222,138,6256,5,1,0,0,0,0,1,0,0


In [215]:
modelbench(DecisionTreeClassifier(), X_train, y_train)

ValueError: multiclass format is not supported

In [ ]:
def modelbench(classifier, X_train, Y_train):
    
    rs = ShuffleSplit(n_splits=3, test_size=.33)
    
    rs.get_n_splits(X_train)
    
    Y_train = Y_train.values
    Y_train = Y_train.ravel()

    for train_idx, test_idx in rs.split(X_train):
        ssX_train, ssX_test = X_train[train_idx], X_train[test_idx]
        ssy_train, ssy_test = Y_train[train_idx], Y_train[test_idx]

        t0 = time.time()
        
        clf = classifier
        clf.fit(X=ssX_train, y=ssy_train)

        ypred = clf.predict(ssX_test)

        scores = roc_auc_score(ssy_test, ypred)
    
        t1 = time.time()
    
        print(f"{classifier} \n took {t1-t0} seconds to run and got a score of {scores}")
        print()

In [ ]:
modelbench(DecisionTreeClassifier(), X_train, y_train)

In [190]:
modelbench(LogisticRegression(), X_train1, y_train1)

KeyError: '[115079   9076 224787 ... 213510  38484 374709] not in index'

In [ ]:
X_train1.head()

In [ ]:
pd.DataFrame(X_train1).describe()

In [ ]:
pd.DataFrame(X_test1).describe()

### RandomForest

In [195]:
params = {
    "max_depth": [2,4,6,8,10],
    "min_samples_split": [5, 9, 19],
    "min_samples_leaf": [2, 5, 10],
    "max_leaf_nodes": [15, 20, 50],
}

In [196]:
grid = GridSearchCV(RandomForestClassifier(n_estimators=10), params, cv=5)

In [197]:
grid

GridSearchCV(cv=5, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'max_depth': [2, 4, 6, 8, 10], 'min_samples_split': [5, 9, 19], 'min_samples_leaf': [2, 5, 10], 'max_leaf_nodes': [15, 20, 50]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [198]:
grid.fit(X_train1, y_train1.values.ravel())

GridSearchCV(cv=5, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'max_depth': [2, 4, 6, 8, 10], 'min_samples_split': [5, 9, 19], 'min_samples_leaf': [2, 5, 10], 'max_leaf_nodes': [15, 20, 50]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [199]:
grid.best_estimator_

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=10, max_features='auto', max_leaf_nodes=50,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=2, min_samples_split=5,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [201]:
grid.best_params_

{'max_depth': 10,
 'max_leaf_nodes': 50,
 'min_samples_leaf': 2,
 'min_samples_split': 5}